# OpenGridMap Notebook  -- Algorithm Inference

Before executing any code in this Notebook, you should 
1. Install what specified in Installation.ipynb
2. Understand DataPreparation.ipynb is also helpful

The goal of this notebook is to
1. Learn APIs of NetworkX
2. Inference and test with different algorithms

# Step1. Data Preparation

## Import necessary modules

In [6]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from networkx.algorithms import approximation as approx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
import math
import matplotlib.pyplot as plt
%matplotlib inline


def cleanMap(transJson, vornoiJson): 
    # Mark transformer
    tCenter = transJson['coordinates']
    tMarker =  Marker(location=tCenter)
    # Mark polygon
    tPoly = Polygon(locations=vornoiJson['coordinates'], weight=2,
                color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
                fill_color='#ccffcc') 
    
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=tCenter ,zoom=16)
    tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

def connect(broken):# Set connection to gis db
    if broken:
        conn.close()
    try:
        conn = psycopg2.connect("dbname=gis user=jennyzhou")
    except:
        print 'Fail to connect to Postgres Server'
    return conn

## Database connection

In [7]:
conn = connect(False)

## Select transformer, house within one vornoi area

In [8]:
cur = conn.cursor()
vid = 38
# Select vornoi polygon and transformer
cur.execute("SELECT ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(way, 4326)))), "
            "ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(tway, 4326)))) FROM vornoi WHERE vid = %d ;" % vid)
data = cur.fetchall()

# Select all the houses
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) "
            "FROM vornoi_map where vid = %d and istransformer = false ;" % vid)
houses = cur.fetchall()
cur.close()

vornoiJson = json.loads(data[0][0])
transJson = json.loads(data[0][1])
houseJson = json.loads(houses[0][0])

# Step2. Data visualisation with ipyleaflet

In [336]:
tMap = cleanMap(transJson, vornoiJson)
tMap

In [200]:
# Mark houses
hmarkers = []
for hp in houseJson['coordinates'][0:1]:
    c = Circle(location= hp, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    hmarkers.append(c)
    tMap.add_layer(c) 

In [198]:
for mk in hmarkers:
    tMap.remove_layer(mk)

## Locating the roads

In [24]:
#conn = connect(False)
# Select roads near one house
cur = conn.cursor()
# cur.execute("SELECT ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM planet_osm_line WHERE "
#             "(ST_Crosses(way,(SELECT way FROM vornoi where vid = %d)) = true " 
#             "OR ST_Within(way,(SELECT way FROM vornoi where vid = %d)) = true)"
#             "AND highway in ('residential');" % (vid,vid))

# ST_ClosestPoint(geometry g1, geometry g2)
# Large road: 'residential'; Small road: 'footway'; Park road: 'service'; 'step' same as 'track'
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(t1.way, 4326)))), "
            "ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(t2.way, 4326))))) "
            "FROM (SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false LIMIT 2) AS t1, "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "
            "WHERE ST_DWithin(t1.way, t2.way, 60) = true;" % (vid,vid))
road_hs = cur.fetchall()

### Select roads of this vornoi 
# cur.execute("SELECT ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM planet_osm_line "
#             "WHERE ST_DWithin((SELECT way FROM vornoi WHERE vid = %d), way, 1) = true "
#             "AND highway in('residential');"% vid)# ")
# roads = cur.fetchall()
# roadVorJson = json.loads(roads[0][0])

### Main street from Garching
# cur.execute("SELECT ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM planet_osm_line "
#             "WHERE ST_Within(way,(SELECT way FROM planet_osm_polygon WHERE osm_id = -30971 )) = true "
#             "AND highway in ('residential');")# in('primary','secondary','primary_link');")
cur.close()
hJson = json.loads(road_hs[0][0])
roadJson = json.loads(road_hs[0][1])

In [35]:
# SELE line_interpolate_point(linestring, line_locate_point(LineString, Point))
# cur = conn.cursor()
# cur.execute("SELECT ST_AsText(ST_Intersection("
#             "ST_GeographyFromText('SRID=4326;LINESTRING(48.2523389116623 11.6491627006109,48.2522655791162 11.6491680905026)'),"
#             "ST_GeographyFromText('SRID=4326;LINESTRING(48.2522697063177 11.6491294629454,48.2522521806625 11.6492982563873)')));")
# ins = cur.fetchall()
# cur.close()
print len(hJson['coordinates'])
print len(roadJson['coordinates'])
for rd in roadJson['coordinates']:
    print rd

3
3
[[48.2528591749442, 11.6464178982603], [48.2528429055817, 11.646492817755], [48.2528036078721, 11.6466796673341], [48.2527902693733, 11.6467524308721], [48.2527756747787, 11.6468755898975], [48.2527582091108, 11.6470348611974], [48.252746365949, 11.6472310532554], [48.2527399658554, 11.6474395522329], [48.2527357788778, 11.6475727723895], [48.2527259095722, 11.6478583468183], [48.2527238160829, 11.647917276301], [48.2527222011053, 11.6479752176368], [48.2527110158892, 11.6483764052427], [48.2527077859333, 11.6484819572886], [48.252696899784, 11.6488343663745], [48.252689901544, 11.6490638859296], [48.2526799724155, 11.649381350551]]
[[48.2505254853757, 11.6473775684783], [48.2505482754774, 11.6474081111979], [48.2505794996947, 11.6474221249164], [48.2506051609624, 11.6474118841221], [48.2506789146043, 11.6473827787069], [48.2507466865108, 11.64738942624], [48.2507697755952, 11.6473915821967], [48.2509937870259, 11.6474518591523], [48.2511658175679, 11.6474981223894], [48.2513630897

In [38]:
tMap = cleanMap(transJson, vornoiJson)
pylines = []
for hs in hJson['coordinates']:
    hc = Circle(location=hs, weight=5, opacity = 0.5, color = '#00134d', radius = 2) 
    pylines.append(hc)
    tMap.add_layer(hc)

#for rd in roadJson['coordinates']:
#for rd in x:
pl = Polyline(locations=roadJson['coordinates'][2], weight=3,color='red', opacity=0.8) 
pylines.append(pl)
tMap.add_layer(pl) 
    
# for rd in roadVorJson['coordinates']:
# #for rd in x:
# #rd = [[48.2522697063177, 11.6491294629454],[48.2522521806625, 11.6492982563873]]
#     pl = Polyline(locations=rd, weight=3,color='blue', opacity=0.8) 
#     pylines.append(pl)
#     tMap.add_layer(pl) 
tMap

In [334]:
for mk in pylines:
    tMap.remove_layer(mk)


# Step3. Construct network graph

## Preparing vertices and edges

In [5]:
cur = conn.cursor()
# Select vornoi polygon and transformer
cur.execute("SELECT oid FROM vornoi_map WHERE vid = %d and istransformer = True;"%vid)
tid = cur.fetchall()[0][0]
cur.execute("SELECT t1.oid, t2.oid, ST_Distance(t1.way, t2.way) "
            "FROM (SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t1, "
            "(SELECT oid, way FROM vornoi_map WHERE vid = %d and istransformer = False) AS t2 "
            "WHERE t1.oid != t2.oid ;"% (vid,vid))
nodes = cur.fetchall()
cur.close()

In [79]:
def saveGraph(conn, vid, edgeList, name):
    # Store result into database
    cur = conn.cursor()
    # Check wheter already stored
    cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326)))"
                "FROM graph WHERE type = '%s' AND vid = %d;" %(name,vid))
    lines = cur.fetchall()
    # If not, store minimum spanning tree into database
    if len(lines)==0:
        print 'Storing topology of %s into db'%name
        cur.execute("CREATE TABLE IF NOT EXISTS templines (way geometry(LineString,900913));")
        conn.commit()   
        # clean entries
        cur.execute("DELETE FROM templines;")
        conn.commit()   

        for edge in edgeList:
            (start, end) = edge
            cur.execute("INSERT INTO templines (SELECT ST_MakeLine(t1.way, t2.way) "
                        "FROM (SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t1, "
                        "(SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t2 "
                        "WHERE t1.oid = %d AND t2.oid = %d);"% (vid, vid, start, end ))
            conn.commit()   

        cur.execute("INSERT INTO graph (vid, type, way) SELECT %d, '%s', ST_Collect(way) FROM templines;"%(vid, name))
        conn.commit()
        cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326)))"
                    "FROM graph WHERE type = '%s' AND vid = %d;" %(name,vid))
        lines = cur.fetchall()
    cur.close()
    return lines

## Network Simplex 

In [80]:
# Build  directed graph
graph = []
for ele in nodes:
    w = int(math.floor(ele[2]))
    graph.append("%d %d %d" % (ele[0],ele[1],w))
G_ = nx.parse_edgelist(graph, create_using = nx.DiGraph(), nodetype = int, data=(('weight',int),))
assert G_.has_node(tid)

for n in G_.nodes(data=True):
    n[1]['demand'] = 1
G_.add_node(tid, demand = -len(G_.nodes())+1)
flowCost, flowDict = nx.capacity_scaling(G_)
#flowCost, flowDict = nx.network_simplex(G_)
NS = sorted([(u, v) for u in flowDict for v in flowDict[u] if flowDict[u][v] > 0])

print approx.node_connectivity(G_), len(NS)
lines = saveGraph(conn, vid, NS, 'NS')

90 90
Storing topology of NS into db


## Minimum Spanning Tree

In [9]:
graph = []
for ele in nodes:
    graph.append("%d %d %f" % ele)
G = nx.parse_edgelist(graph, nodetype = int, data=(('weight',float),))

MST = nx.minimum_spanning_tree(G)
print approx.node_connectivity(G), len(MST.nodes())
#lines = saveGraph(conn, vid, MST.edges(), 'MST')

90 91


## Minimum Spanning Arborescence 

In [84]:
graph = []
for ele in nodes:
    graph.append("%d %d %f" % ele)
G_ = nx.parse_edgelist(graph, create_using = nx.DiGraph(), nodetype = int, data=(('weight',float),))
assert G_.has_node(tid)

MSA = nx.minimum_spanning_arborescence(G_)
print approx.node_connectivity(G_), len(MSA.nodes())

lines = saveGraph(conn, vid, MSA.edges(), 'MSA')

90 91
Storing topology of MSA into db


## Minimum Branching 

In [89]:
# Not applicable
# graph = []
# for ele in nodes:
#     graph.append("%d %d %f" % ele)
# G_ = nx.parse_edgelist(graph, create_using = nx.DiGraph(), nodetype = int, data=(('weight',float),))
# assert G_.has_node(tid)

# MB = nx.minimum_branching(G_)
# print approx.node_connectivity(G_),len(MB.nodes())
# lines = saveGraph(conn, vid, MB.edges(), 'MB')
# print len(res[0][0])


# Step4. Display graph on map

In [87]:
tmpJson = json.loads(lines[0][0])
lmarkers = []

tmpMap = cleanMap(transJson, vornoiJson)
for l in tmpJson['coordinates']:
    pl = Polyline(locations=l, weight=2, color='#2929a3', opacity=0.5)
    lmarkers.append(pl)
    tmpMap.add_layer(pl)
tmpMap